# Marine Biodiversity and Environmental Data Exploration with `mlcroissant`
This notebook provides a guide for loading and exploring the Marine Biodiversity and Environmental Data dataset using the `mlcroissant` library.

### Dataset Source
The dataset source is provided via a Croissant schema URL.

In [ ]:
# Ensure `mlcroissant` library is installed
!pip install mlcroissant

## 1. Data Loading
Load metadata and records from the dataset using `mlcroissant`.

In [ ]:
import mlcroissant as mlc
import pandas as pd

# Define the dataset URL
url = 'https://cdn.dev.senscience.cloud/portals/10.82843/pm80-mh77/fair2.json'

# Load the dataset metadata
dataset = mlc.Dataset(url)
metadata = dataset.metadata.to_json()
print(f"{metadata['name']}: {metadata['description']}")

## 2. Data Overview
Review available record sets, fields, and their IDs.

In [ ]:
# Display the available record sets
record_sets = metadata['recordSet']
for record_set in record_sets:
    print(f"Record Set @id: {record_set['@id']}")
    for field in record_set['field']:
        print(f" - Field Name: {field['name']}, Field @id: {field['@id']}")

## 3. Data Extraction
Load data from a specific record set into a DataFrame for analysis. Use the record set and field `@id`s from the overview.

In [ ]:
# Extract data from each record set
record_set_ids = [
    'https://senscience.ai/frontiers/borja/README.csv',
    'https://senscience.ai/frontiers/borja/METADATA.csv',
    'https://senscience.ai/frontiers/borja/WATER.csv'
]
dataframes = {}

for record_set_id in record_set_ids:
    records = list(dataset.records(record_set=record_set_id))
    dataframes[record_set_id] = pd.DataFrame(records)

# Display column names of a specific record set
print(dataframes['https://senscience.ai/frontiers/borja/README.csv'].columns.tolist())
dataframes['https://senscience.ai/frontiers/borja/README.csv'].head()

## 4. Exploratory Data Analysis (EDA)
Apply common data processing steps, such as filtering records based on specific criteria, normalizing numeric fields, and categorizing data. This section should include operations like removing outliers, transforming data distributions, or grouping data by key attributes to prepare it for further analysis.

In [ ]:
# Select a numeric field for analysis
numeric_field = 'https://api.dev.senscience.cloud/frontiers/8704cfcf-0667-4eaf-b0b1-679597df683f-16/d2d0459d-a81e-4bcc-9438-3b3cae9c4fb5'

threshold = 2000
filtered_df = dataframes['https://senscience.ai/frontiers/borja/METADATA.csv'][dataframes['https://senscience.ai/frontiers/borja/METADATA.csv'][numeric_field] > threshold]
print(f"Filtered records with {numeric_field} > {threshold}:")
print(filtered_df.head())

# Normalize the numeric field for filtered records
filtered_df[f"{numeric_field}_normalized"] = (filtered_df[numeric_field] - filtered_df[numeric_field].mean()) / filtered_df[numeric_field].std()
print(f"Normalized {numeric_field} for filtered records:")
print(filtered_df[[numeric_field, f"{numeric_field}_normalized"]].head())

# Group data by a specific field
group_field = 'https://api.dev.senscience.cloud/frontiers/8704cfcf-0667-4eaf-b0b1-679597df683f-16/bdf3b9e8-ec83-42a0-8136-071e7e298518'
if group_field in dataframes['https://senscience.ai/frontiers/borja/METADATA.csv'].columns:
    grouped_df = filtered_df.groupby(group_field).mean()
    print(f"Grouped data by {group_field}:")
    print(grouped_df.head())

## 5. Visualization
Visualize data distributions or relationships between fields in the dataset.

In [ ]:
import matplotlib.pyplot as plt

# Plot the distribution of the normalized numeric field
plt.hist(filtered_df[f"{numeric_field}_normalized"], bins=30, alpha=0.7, color='blue')
plt.title(f'Distribution of {numeric_field}_normalized')
plt.xlabel('Normalized Value')
plt.ylabel('Frequency')
plt.show()

## 6. Conclusion
Summarize key findings and observations from the dataset exploration.

In this notebook, we explored the Marine Biodiversity and Environmental Data using `mlcroissant`. We extracted metadata, examined various record sets, and performed exploratory data analysis. Through data filtering, normalization, and visualization, we gained insights into the dataset's structure and content. This analysis can serve as a basis for more in-depth research or model building.